In [1]:
import numpy as np
import os
import time
from vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.applications.imagenet_utils import decode_predictions
from keras.layers import Dense, Activation, Flatten
from keras.layers import merge, Input
from keras.models import Model
from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import skimage

import csv
import constant
from pathlib import Path

Using TensorFlow backend.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15429695163308964511
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1450105241
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5114541358264095914
physical_device_desc: "device: 0, name: GeForce GTX 950M, pci bus id: 0000:01:00.0, compute capability: 5.0"
]






### Returns dictionary with image paths as key and labels as values

In [2]:
def get_labels(file='metadata.csv'):
    
    return_dict = {}
    
    with open(file, 'r') as metadata:
        lines = csv.reader(metadata)
        dataset = list(lines)

        # for x in range(len(dataset)):
        for x in range(constant.INPUT_SIZE):
            img_path = constant.DATASET_DIR + dataset[x][1] + '.jpg'
            path_obj = Path(img_path)
            if not path_obj.is_file():
                continue
            if dataset[x][2] == 'nv':
                return_dict[img_path] = 0
            elif dataset[x][2] == 'mel':
                return_dict[img_path] = 1
            
    print("Dictionary length :", len(return_dict), end="\n\n")
    # print("Random value :", return_dict['ISIC_0024306'])
                
    return return_dict

### Returns lists of images and labels in proper order

In [5]:
def load_data():

    labels = []
    images = []
    
    labels_dict = get_labels()
    
    for key, value in labels_dict.items():
        # images.append(skimage.data.imread(key))
        labels.append(value)
        
        img = image.load_img(key, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        
        # print('Input image shape:', x.shape)
        
        images.append(x)
        
    return images, labels

In [10]:
    # Loading data
    images,labels = load_data()

Dictionary length : 2134



In [11]:
# Prepairing data
img_data = np.array(images)
print (img_data.shape)
img_data=np.rollaxis(img_data,1,0)
print (img_data.shape)
img_data=img_data[0]
print ("Final shape :", img_data.shape) 

classes = 2
samples = img_data.shape[0]
print("Number of images :", samples)

Y = np_utils.to_categorical(labels, classes)

print("Shuffling...")
x,y = shuffle(img_data, Y, random_state=2)
print("Done")
print("Splitting...")
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)
print("Done")

(2134, 1, 224, 224, 3)
(1, 2134, 224, 224, 3)
Final shape : (2134, 224, 224, 3)
Number of images : 2134
Shuffling...
Done
Splitting...
Done


In [12]:
# Loading initial model which is VGG-16 
image_input = Input(shape=(224,224,3))

model = VGG16(input_tensor=image_input, include_top=True, weights='imagenet')

model.summary()







Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       

In [13]:
# Modifying network by changing last fully connected layer
last_layer = model.get_layer("fc2").output
out = Dense(classes, activation='softmax', name='output')(last_layer)
new_model = Model(image_input, out)
new_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [14]:
# Freezing all except last layer
for layer in new_model.layers[:-1]:
    layer.trainable = False

new_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [15]:
# Compiling model
new_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [ ]:
# Training
print("Training...")
start_time = time.time()
hist = new_model.fit(X_train, y_train, batch_size=32, epochs=12, verbose=2, validation_data=(X_test, y_test))
print('Training time : {:.2f}'.format(start_time - time.time()))

model.save_weights('trained_model.h5')

In [ ]:
# Validation
(loss, accuracy) = new_model.evaluate(X_test, y_test, batch_size=10, verbose=2)

print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))

In [18]:
# Modifying network by changing all fully connected layers

image_input = Input(shape=(224, 224, 3))

model = VGG16(input_tensor=image_input, include_top=True,weights='imagenet')

model.summary()

last_layer = model.get_layer('block5_pool').output
x = Flatten(name='flatten')(last_layer)
x = Dense(128, activation='relu', name='fc1')(x)
x = Dense(128, activation='relu', name='fc2')(x)
output = Dense(classes, activation='softmax', name='output')(x)
modified_model = Model(image_input, output)
modified_model.summary()

# Freeze layers that are not fully connected
for layer in modified_model.layers[:-3]:
    layer.trainable = False

modified_model.summary()

modified_model.compile(loss='categorical_crossentropy',optimizer='adadelta',metrics=['accuracy'])

start_time=time.time()
hist = modified_model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=2, validation_data=(X_test, y_test))
print('Training time : {:.2f}' % (start_time - time.time()))
(loss, accuracy) = modified_model.evaluate(X_test, y_test, batch_size=10, verbose=2)

print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

Train on 1707 samples, validate on 427 samples
Epoch 1/10


KeyboardInterrupt: 